# Ejemplo de Spark SQL

El objetivo de este notebook es presentar un ejemplo sobre el API de SQL, aplicando queries sobre los archivos <code>flights.csv</code>

In [1]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("App_spark_Sql").getOrCreate()

In [6]:
flights = spark.read.option("header",True).csv("./data/flights.csv")

Para el uso de la API estructurada, es primordial registrar todos los DataFrames que queramos usar commo tablas o vistas

In [7]:
flights.createOrReplaceTempView("flights")

Calcular los vuelos que llegan a cada aeropuerto de destino usando sentencia SQL 

In [8]:
fl_dest_count = spark.sql("""SELECT dest,count(dest) as dest_count 
                            FROM  flights GROUP BY dest ORDER BY dest_count DESC""")

In [9]:
fl_dest_count

DataFrame[dest: string, dest_count: bigint]

Nuevamente registramos el resultado como una vista. 

In [10]:
fl_dest_count.createOrReplaceTempView("flights_dest_count")

In [11]:
fl_dest_count.show(5)

+----+----------+
|dest|dest_count|
+----+----------+
| ORD|     17283|
| ATL|     17215|
| LAX|     16174|
| BOS|     15508|
| MCO|     14082|
+----+----------+
only showing top 5 rows



Ahora cargaremos los datos del archivo de aeropuertos 

In [12]:
airports_df = spark.read.option("header",True).csv("./data/airport-codes.csv")

In [13]:
airports_df.createOrReplaceTempView("airports")

Ejecutar una Unión (join) en Spark SQL  es muy similar a SQL:

In [15]:
sql ="""SELECT
            a.name,
            f.dest_count 
        FROM   flights_dest_count f 
        JOIN airports a on a.iata_code = f.dest
        ORDER BY dest_count DESC
"""

fl_dest_airports = spark.sql(sql)

In [17]:
fl_dest_airports.show(10)

+--------------------+----------+
|                name|dest_count|
+--------------------+----------+
|Chicago O'Hare In...|     17283|
|Hartsfield Jackso...|     17215|
|Los Angeles Inter...|     16174|
|General Edward La...|     15508|
|Orlando Internati...|     14082|
|Charlotte Douglas...|     14064|
|San Francisco Int...|     13331|
|Fort Lauderdale H...|     12055|
|Miami Internation...|     11728|
|Ronald Reagan Was...|      9705|
+--------------------+----------+
only showing top 10 rows

